In [42]:
from pathlib import Path
import time
import json
import shutil
import cv2
from collections import defaultdict
import random

In [43]:

main_dir = Path('/home/inonzr/datasets/samsung')
from_dir = main_dir / 'crops'
to_dir = main_dir / time.strftime("%Y%m%d_%H%M%S")
metadata_dir= main_dir / 'metadata.json'

to_dir.mkdir(parents=True, exist_ok=True)
if from_dir.exists():
    print("folder connected")

imeges_count = 1576

folder connected


In [44]:

def crop_img(img_dir,save_dir,img_name):
    img = cv2.imread(str(img_dir))
    h,w = img.shape[:2]
    crop_size = 50
    start_y = (h-crop_size)//2
    start_x = (w-crop_size)//2
    crop = img[start_y:start_y+crop_size,start_x:start_x+crop_size]
    save_path = save_dir / f"{img_name}.png"
    cv2.imwrite(str(save_path), crop)


In [45]:
distortion_counts = defaultdict(int)

with open(metadata_dir, 'r') as f:
    metadata = json.load(f)
count = 0
not_exist = 0
for entry in metadata:
    for img_id, data in entry.items():
        artifact = data['artifact']
        size = data['size']
        if size <= 60:
            distortion_counts[artifact] += 1

for i in distortion_counts:
    print(f"{i}: {distortion_counts[i]}")

good_arr = []
for i in distortion_counts:
    if distortion_counts[i] >=imeges_count:
        good_arr.append(i)

Smears: 2905
Saturated False Color: 1576
Over Desaturation: 3196
Corner: 266
Zagging: 250
Broken Line: 4437
Moire: 1054
Edge False Color: 4342
Line Noise: 152
Pattern artifact: 30


In [29]:
distortion_counts = defaultdict(int)

with open(metadata_dir, 'r') as f:
    metadata = json.load(f)
count = 0
not_exist = 0
for entry in metadata:
    for img_id, data in entry.items():
        artifact = data['artifact']
        highlighting = data['highlighting']
        size = data['size']
        if size <= 60 and artifact in good_arr:
            if distortion_counts[artifact] >= imeges_count:
                continue
            artifact_dir = to_dir / artifact
            artifact_dir.mkdir(parents=True, exist_ok=True)
            img_dir = from_dir/f"{img_id}.png"
            if (from_dir/f'{img_id}.png').exists():
                count += 1
                if size == 50:
                        shutil.copy2(img_dir , artifact_dir)
                elif size == 60:
                        crop_img(img_dir,artifact_dir,img_id)
            else:
                    not_exist += 1


print(f'exist: {count}')
print(f'not exist: {not_exist}')

exist: 16456
not exist: 0


In [46]:
distortion_counts = defaultdict(int)
imeges_count = 1405

with open(metadata_dir, 'r') as f:
    metadata = json.load(f)

count = 0
not_exist = 0

for entry in metadata:
    for img_id, data in entry.items():
        artifact = data['artifact']
        size = data['size']

        if size <= 60 and artifact in good_arr:

            if distortion_counts[artifact] >= imeges_count:
                continue

            img_path = from_dir / f"{img_id}.png"

            if img_path.exists():
                artifact_dir = to_dir / artifact
                artifact_dir.mkdir(parents=True, exist_ok=True)

                if size == 50:
                    shutil.copy2(img_path, artifact_dir)
                    distortion_counts[artifact] += 1
                    count += 1
                elif size == 60:
                    crop_img(img_path, artifact_dir, img_id)
                    distortion_counts[artifact] += 1
                    count += 1
            else:
                not_exist += 1

print(f'Total images processed: {count}')
print(f'Missing images: {not_exist}')

for art, c in distortion_counts.items():
    print(f"Artifact {art}: {c} images")

Total images processed: 7025
Missing images: 0
Artifact Smears: 1405 images
Artifact Saturated False Color: 1405 images
Artifact Over Desaturation: 1405 images
Artifact Broken Line: 1405 images
Artifact Edge False Color: 1405 images


#### Do No Run!!


In [ ]:
from pathlib import Path

def limit_images_in_dir(target_dir, max_images=1564):
    target_path = Path(target_dir)

    all_images = list(target_path.glob("*.png")) + list(target_path.glob("*.jpg"))
    num_images = len(all_images)

    if num_images <= max_images:
        print(f"Folder '{target_path.name}' already contains {num_images} images. Nothing to delete.")
        return

    to_delete_count = num_images - max_images
    print(f"Found {num_images} images in '{target_path.name}'. Deleting {to_delete_count} excess images...")

    for i in range(max_images, num_images):
        all_images[i].unlink()

    print(f"Done! Exactly {max_images} images remaining in '{target_path.name}'.")


normal_folder = main_dir / 'all_crops'
if normal_folder.exists():
    limit_images_in_dir(normal_folder, 7025)

for artifact_name in good_arr:
    artifact_folder = main_dir / '20260219_174631' / artifact_name
    if artifact_folder.exists():
        limit_images_in_dir(artifact_folder, 1564)

In [ ]:
import zipfile
from pathlib import Path

# הגדרת הנתיבים
zip_path = Path('/home/inonzr/datasets/samsung/Fuziki.zip')
extract_to = Path('/home/inonzr/datasets/samsung/Fuziki')

# חילוץ הקובץ
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
    print(f"Successfully extracted to: {extract_to}")

In [49]:
import cv2
import os
from pathlib import Path

# הגדרת נתיבים
src_folder = Path('/home/inonzr/datasets/samsung/Fuziki/Fuziki')
dst_root = Path('/home/inonzr/datasets/samsung/all_crops')
dst_root.mkdir(parents=True, exist_ok=True)

crop_size = 50

# 1. מעבר על כל הקבצים בתיקיית המקור
for img_path in src_folder.glob('*.bmp'):
    print(f"Processing: {img_path.name}")

    # טעינת התמונה
    img = cv2.imread(str(img_path))
    if img is None:
        continue

    h, w = img.shape[:2]

    # חילוץ שם הקובץ ללא סיומת (למשל Fuziki_case2)
    base_name = img_path.stem

    count = 0
    # 2. לולאות שרצות על הגובה והרוחב
    for y in range(0, h - crop_size + 1, crop_size):
        for x in range(0, w - crop_size + 1, crop_size):
            # ביצוע החיתוך
            crop = img[y:y + crop_size, x:x + crop_size]

            # הוספת שם המקור לשם ה-Crop כדי למנוע דריסה
            # השם ייראה כך: Fuziki_case2_crop_100_150.png
            crop_name = f"{base_name}_crop_{y}_{x}.png"

            # שמירה ישירות בתיקיית היעד הראשית
            cv2.imwrite(str(dst_root / crop_name), crop)
            count += 1

    print(f"Finished {img_path.name}: Created {count} crops in {dst_root.name}")

print(f"\nAll done! All crops are located in: {dst_root}")

Processing: Fuziki_case19_Regular@ref.bmp
Finished Fuziki_case19_Regular@ref.bmp: Created 7448 crops in all_crops
Processing: Fuziki_case2_Regular@ref.bmp
Finished Fuziki_case2_Regular@ref.bmp: Created 7448 crops in all_crops
Processing: Fuziki_case10_Regular@new.bmp
Finished Fuziki_case10_Regular@new.bmp: Created 7448 crops in all_crops
Processing: Fuziki_case2_Regular@new.bmp
Finished Fuziki_case2_Regular@new.bmp: Created 7448 crops in all_crops
Processing: Fuziki_case1_Regular@ref.bmp
Finished Fuziki_case1_Regular@ref.bmp: Created 7448 crops in all_crops
Processing: Fuziki_case3_Regular@new.bmp
Finished Fuziki_case3_Regular@new.bmp: Created 7448 crops in all_crops
Processing: Fuziki_case4_Regular@gt.bmp
Finished Fuziki_case4_Regular@gt.bmp: Created 7448 crops in all_crops
Processing: Fuziki_case4_Regular@new.bmp
Finished Fuziki_case4_Regular@new.bmp: Created 7448 crops in all_crops
Processing: Fuziki_case14_Regular@ref.bmp
Finished Fuziki_case14_Regular@ref.bmp: Created 7448 crops i

In [50]:
import shutil
from pathlib import Path

# הנתיב לתיקייה שאתה רוצה לדחוס (ה-Crops שיצרת)
dir_to_zip = Path('/home/inonzr/datasets/samsung/20260219_115908')

# השם והנתיב של קובץ ה-ZIP שייווצר (אין צורך להוסיף .zip בסוף, הוא מוסיף לבד)
output_filename = '/home/inonzr/datasets/samsung/20260219_115908'

# ביצוע הדחיסה
# format='zip' - סוג הקובץ
# root_dir - התיקייה שדוחסים
shutil.make_archive(output_filename, 'zip', dir_to_zip)

print(f"Done! Zip file created at: {output_filename}.zip")

Done! Zip file created at: /home/inonzr/datasets/samsung/20260219_115908.zip
